## Calculating the slope bricks needed for a square roof


### The algorithm for calculating the peg count of a square roof

If you start with a square roof that's 8 pegs wide on a side, the next level will be 6, then 4, and finally the top is 2. The following algorithm expresses the total length. "Bw" is the base width of the roof side.

\begin{equation}
\sum_{n=Bw}^{2} n-2
\end{equation}

### Calculate the number of pegs per side

In [1]:
def peg_width_per_levels(base_width):
    limiter = 2
    decrementer = -2
    decrementing_width = int(base_width)
    peg_count_per_level = []
    while decrementing_width >= limiter:
        peg_count_per_level.append(int(decrementing_width))
        decrementing_width += decrementer
    return peg_count_per_level

### Calculate the number of pegs in the perimeter at a level

In [2]:
def peg_perimeter(side_lengths):
    return sum(side_lengths) - len(side_lengths)

### Calculate the needed slope bricks per level

In [8]:
def slope_bricks_per_level_per_side(side_length):
    # Get the number of 4x2 slope bricks needed.
    four_brick_count = int(side_length / 4)
    remainder_bricks = side_length % 4 if side_length > 0 else 0
    # Calculate the remainder bricks you need.
    remainder_two_bricks = 1 if remainder_bricks > 1 else 0
    remainder_one_bricks = remainder_bricks % 2
    return four_brick_count, remainder_two_bricks, remainder_one_bricks

def slope_bricks_per_level(side_lengths):
    # If the sum of the sides minus the sides is not greater than the sides there's no need for corner bricks
    # TODO: Define a better condition for calculating corners.
    # TODO: Account for 3x3 corners and other slope angles.
    corner_bricks = len(side_lengths) if peg_perimeter(side_lengths) > len(side_lengths) else 0
    four_bricks = 0; two_bricks = 0; one_bricks = 0
    for side_length in side_lengths:
        # Each side has four pegs dedicated to corner slopes.
        sans_corners = side_length - 4
        four_bricks, two_bricks, one_bricks = tuple(map(sum, zip((four_bricks, two_bricks, one_bricks),
                                                                 slope_bricks_per_level_per_side(sans_corners))))
    return corner_bricks, int(four_bricks), int(two_bricks), int(one_bricks)

### Calculate the amount of slope bricks needed in a square roof

In [9]:
def bricks_per_square_roof(base_width):
    corner_bricks = 0; four_bricks = 0; two_bricks = 0; one_bricks = 0
    for level_length in peg_width_per_levels(base_width):
        four_sides = [level_length for x in range(4)]
        corner_bricks, four_bricks, two_bricks, one_bricks = tuple(map(sum, zip(slope_bricks_per_level(four_sides),
                                                                               (corner_bricks, four_bricks, 
                                                                                two_bricks, one_bricks))))
    return corner_bricks, four_bricks, two_bricks, one_bricks

### Output the bricks needed to build a square roof

In [10]:
from IPython.core.display import display, HTML
from ipywidgets.widgets import IntText, Output

In [11]:
def display_needed_bricks(base_width):
    header = f'<tr><td colspan=\"2\" style=\"text-align:center\">Base width: {base_width}</td></tr>'
    labels = ("2x2 Corner", "4x2 Slope", "2x2 Slope", "1x2 Slope")
    bricks = bricks_per_square_roof(base_width)
    rows = [f"<tr>{c}</tr>" for c in [f"<td>{l}</td><td>{b}</td>" for l, b in zip(labels, bricks)]]
    display(HTML(f"<table>{header}{''.join(rows)}</table>"))

### Choose your size

In [12]:
int_text = IntText(description="Base width:")
out_bricks = Output()

display(int_text, out_bricks)

def on_value_change(change):
    out_bricks.clear_output()
    with out_bricks:
        display_needed_bricks(int(change["new"]))
    
int_text.observe(on_value_change, names='value')

IntText(value=0, description='Base width:')

Output()